In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
root_path = '../'
sys.path.insert(0, root_path)

In [ ]:
import os
from natsort import natsorted

from scripts.create_problem import load_problem_set
from scripts.run_solver import load_results
from utils import *

In [ ]:
read_dir = 'data/test'
# read_dir = "data/various"
# read_dir = "data/normal"
# read_dir = "data/distributions"
problem_list = natsorted(os.listdir(read_dir))

# problem_list = [
#     'SPP',
#     'GraphColoring',
#     'Knapsack',
#     'MVC',
#     'QUBO',
#     'SetPack',
#     'WMaxCut',
#     'NumberPartitioning',
#     'MaxCut',
#     'WMVC',
#     'Ising',
#     'Max2Sat',
#     'QAP'
# ]

X, y = get_Xy(read_dir, problem_list, suffix='train')
X_tsne = get_tsne(X)
show_tsne(problem_list, X_tsne, y)
print_unique(read_dir, problem_list)

In [ ]:
# from scripts.create_problem import load_problem_set
# from scripts.run_solver import load_results

# d_x = []
# problem_name = 'QAP'
# problems = load_problem_set(f'{read_dir}/{problem_name}/train')
# for problem in problems:
#     d_x.append(problem.info['x_best'])
#     print(problem.Q)
# x_unique = torch.unique(torch.stack(d_x), dim=0)
# print(x_unique)

In [ ]:
read_dir = 'results/test'
# read_dir = 'results/various'
# read_dir = 'results/normal'
# read_dir = 'results/distributions'

problem_list = natsorted(os.listdir(read_dir))
meta_results = defaultdict(dict)
for problem in problem_list:
    for solver in ('RandomSearch', 'PSO', 'PROTES'):
        meta_results[problem][solver] = get_meta_results(problem, solver, read_dir, suffix='train', budget=1000)

In [ ]:
show_meta_results(meta_results)

In [ ]:
problem_averaged_meta_results = get_problem_averaged_meta_results(meta_results)
show_meta_results(problem_averaged_meta_results)